In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']

# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
%sql
-- Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
from pyspark.sql.types import StructType, StructField, StringType

# Define the schema for each DataFrame to provide the structure to the streaming data
pinSchema = StructType([ 
    StructField("index", StringType(), True), 
    StructField("unique_id", StringType(), True), 
    StructField("title", StringType(), True), 
    StructField("description", StringType(), True), 
    StructField("poster_name", StringType(), True), 
    StructField("follower_count", StringType(), True), 
    StructField("tag_list", StringType(), True), 
    StructField("is_image_or_video", StringType(), True), 
    StructField("image_src", StringType(), True), 
    StructField("downloaded", StringType(), True), 
    StructField("save_location", StringType(), True), 
    StructField("category", StringType(), True)
])

geoSchema = StructType([
    StructField("index", StringType(), True), 
    StructField("timestamp", StringType(), True), 
    StructField("latitude", StringType(), True), 
    StructField("longitude", StringType(), True),
    StructField("country", StringType(), True)
])

userSchema = StructType([
    StructField("index", StringType(), True), 
    StructField("first_name", StringType(), True), 
    StructField("last_name", StringType(), True), 
    StructField("age", StringType(), True),
    StructField("date_joined", StringType(), True)
])

In [0]:


class Streams:
    ''' 
    Streams is a class used to read the streaming data from AWS Kinesis, deserialise it, and converting it to a streaming DataFrame (df) based on chosen schema.
    
    Parameters:
    -------
    stream_name: string
        A name of the stream in Kinesis 

    Methods:
    -------
    load_stream()
        Reads the streaming data from Kinesis and returns a df.
    deserialise_stream()
        Returns a deserialised data column of the df casted as a string.
    convert_to_df(schema)
        Converts deserialised data into a df using from_json() function and chosen schema. Returns a streaming df.
    '''
    def __init__(self, stream_name):
        self.stream_name = stream_name

    def load_stream(self):
        df = spark \
        .readStream \
        .format('kinesis') \
        .option('streamName', self.stream_name) \
        .option('initialPosition','earliest') \
        .option('region','us-east-1') \
        .option('awsAccessKey', ACCESS_KEY) \
        .option('awsSecretKey', SECRET_KEY) \
        .load()
        return df

    def deserialise_stream(self):
        return self.load_stream().selectExpr("CAST(data as STRING)")
         
    
    def convert_to_df(self, schema):
        df = self.deserialise_stream().withColumn("jsonData", from_json(col("data"), schema)).select("jsonData.*")
        return df

pin_stream = Streams('streaming-0e172e8c4bc3-pin')
geo_stream = Streams('streaming-0e172e8c4bc3-geo')
user_stream = Streams('streaming-0e172e8c4bc3-user')

df_pin = pin_stream.convert_to_df(pinSchema)
df_geo = geo_stream.convert_to_df(geoSchema)
df_user = user_stream.convert_to_df(userSchema)


In [0]:
%run Users/kasalotas@yahoo.com/transformation_functions

In [0]:
df_P = clean_pin(df_pin)
df_P.printSchema()
display(df_P)

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,null,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,/data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,124000,Of Life & Lisa | Lifestyle Blog,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,https://i.pinimg.com/originals/b3/bc/e2/b3bce2964e8c8975387b39660eed5f16.jpg,/data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",0,Consuelo Aguirre,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,/data/finance,finance
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,51000,Commitment Connection,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,https://i.pinimg.com/originals/c6/64/ee/c664ee71524fb5a6e7b7b49233f93b43.png,/data/quotes,quotes
8731,ea760f71-febf-4023-b592-d17396659039,20 Koi Fish Tattoos For Lucky Men,"Koi fish tattoos are a popular choice for men who want to make a statement, thanks to their rich symbolism and bold design.",211000,TheTrendSpotter,"Dr Tattoo,Wörter Tattoos,Pisces Tattoos,Tatoo Art,Dream Tattoos,Dope Tattoos,Mini Tattoos,Finger Tattoos,Body Art Tattoos",image,https://i.pinimg.com/originals/8a/0c/0a/8a0c0a7b6236565c519acd41ad1a52c0.jpg,/data/tattoos,tattoos
1313,44662045-e891-4821-8a19-ebe7eedd371a,Liquid Lash Extensions Mascara,"Instantly create the look of lash extensions with this award-winning, best-selling mascara that won't clump, flake or smudge. Available in 3 shades!",43000,Thrive Causemetics,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",video,https://i.pinimg.com/videos/thumbnails/originals/69/84/e2/6984e20f3e262098fa9c0614c3453254.0000001.jpg,/data/beauty,beauty
4315,21b59ba9-829d-4c33-8c27-4cd4c56d26b8,Podcasts for Teachers or Parents of Teenagers,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",25000,Math Giraffe,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",image,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,/data/education,education
10794,c4bd2577-a7bb-4409-bb7a-17d5ed7e1cf1,TireBuyer,Nissan GT-R. Sick.,437,Ray Uyemura,"Lowrider,Old Vintage Cars,Antique Cars,Austin Martin,Nissan Gtr Black,Jaguar,1959 Cadillac,Cadillac Ct6,Old School Cars",image,https://i.pinimg.com/originals/0d/29/9f/0d299f3df020395aa7ce8387f40fbeed.jpg,/data/vehicles,vehicles
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",26000,"Living Low Key | Save Money, Make Money, & Frugal Living","Financial Peace,Financial Tips,Saving Money Quotes,Total Money Ma

In [0]:
df_G = clean_geo(df_geo)
display(df_G)

ind,country,coordinates,timestamp
7528,Albania,"-89.9787, -173.293",2020-08-28T03:52:47.000+0000
2863,Armenia,"-5.34445, -177.924",2020-04-27T13:34:16.000+0000
5730,Colombia,"-77.015, -101.437",2021-04-19T17:37:03.000+0000
8304,French Guiana,"-28.8852, -164.87",2019-09-13T04:50:29.000+0000
8731,Aruba,"-83.104, -171.302",2020-07-17T04:39:09.000+0000
1313,Maldives,"77.0447, 61.9119",2018-06-26T02:39:25.000+0000
4315,Cote d'Ivoire,"-45.8508, 66.1003",2019-12-15T03:51:28.000+0000
10794,Cocos (Keeling) Islands,"-89.5236, -154.567",2022-01-01T02:26:50.000+0000
5494,Bulgaria,"-82.6768, -129.202",2021-07-21T02:02:35.000+0000
5069,Azerbaijan,"-63.0063, -157.474",2021-03-20T09:32:44.000+0000
